# Phase 1: Training Orchestration (Google Colab & Kaggle)

This notebook orchestrates all training activities for **Google Colab or Kaggle execution** with GPU compute support.

## Overview

- **Step 1**: Repository Setup & Environment Configuration
- **Step 2**: Load Centralized Configs
- **Step 3**: Verify Local Dataset (from data config)
- **Step 4**: Setup Local Environment
- **Step 5**: The Dry Run
- **Step 6**: The Sweep (HPO) - Local with Optuna
- **Step 5.5**: Benchmarking Best Trials (NEW)
- **Step 7**: Best Configuration Selection (Automated)
- **Step 8**: Final Training (Post-HPO, Single Run)
- **Step 9**: Model Conversion & Optimization

## Important

- This notebook **executes training in Google Colab or Kaggle** (not on Azure ML)
- All computation happens on the platform's GPU
- **Storage & Persistence**:
  - **Google Colab**: Checkpoints are automatically saved to Google Drive for persistence across sessions
  - **Kaggle**: Outputs in `/kaggle/working/` are automatically persisted - no manual backup needed
- The notebook must be **re-runnable end-to-end**
- Uses the dataset path specified in the data config (from `config/data/*.yaml`), typically pointing to a local folder included in the repository
- **Session Management**:
  - **Colab**: Sessions timeout after 12-24 hours (depending on Colab plan). Checkpoints are saved to Drive automatically.
  - **Kaggle**: Sessions have time limits based on your plan. All outputs are automatically saved.


## Step 1: Repository Setup

Set up the repository in Google Colab or Kaggle. Choose one of the following options:

### Option A: Clone from Git (Recommended)

If your repository is on GitHub/GitLab, clone it:

**For Google Colab:**
```python
!git clone -b feature/google-colab-compute https://github.com/longdang193/resume-ner-azureml.git /content/resume-ner-azureml
```

**For Kaggle:**
```python
!git clone -b feature/google-colab-compute https://github.com/longdang193/resume-ner-azureml.git /kaggle/working/resume-ner-azureml
```

### Option B: Upload Files

**For Google Colab:**
1. Use the Colab file browser (folder icon on left sidebar)
2. Upload your project files to `/content/resume-ner-azureml/`
3. Ensure the directory structure matches: `src/`, `config/`, `notebooks/`, etc.

**For Kaggle:**
1. Use the Kaggle file browser (Data tab)
2. Upload your project files to `/kaggle/working/resume-ner-azureml/`
3. Ensure the directory structure matches: `src/`, `config/`, `notebooks/`, etc.


In [ ]:
!git clone -b feature/google-colab-compute https://github.com/longdang193/resume-ner-azureml.git /content/resume-ner-azureml

### Verify Repository Setup

After cloning or uploading, verify the repository structure:


In [ ]:
from pathlib import Path
import os

# Determine base directory based on environment (will be set in environment detection cell)
# This allows the cell to work even if environment detection hasn't run yet
if "BASE_DIR" not in globals():
    if "COLAB_GPU" in os.environ or "COLAB_TPU" in os.environ:
        BASE_DIR = Path("/content")
    elif "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
        BASE_DIR = Path("/kaggle/working")
    else:
        BASE_DIR = Path("/content")  # Default to Colab path

# Set repository root directory
# Change this if you used a different path in Step 1
ROOT_DIR = BASE_DIR / "resume-ner-azureml"

# Verify repository structure
if not ROOT_DIR.exists():
    raise FileNotFoundError(
        f"Repository not found at {ROOT_DIR}\n"
        f"Please run Step 1 to clone or upload the repository."
    )

required_dirs = ["src", "config", "notebooks"]
missing_dirs = [d for d in required_dirs if not (ROOT_DIR / d).exists()]

if missing_dirs:
    raise FileNotFoundError(
        f"Missing required directories: {missing_dirs}\n"
        f"Please ensure the repository structure is correct."
    )

print(f"✓ Repository found at: {ROOT_DIR}")
print(f"✓ Required directories found: {required_dirs}")


## Step 2: Install Dependencies

Install all required Python packages. PyTorch is usually pre-installed in Colab, but we'll verify the version.


In [ ]:
import torch

# Check PyTorch version (usually pre-installed in Colab)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Visible GPUs: {device_count}")
    for i in range(device_count):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")

# Verify PyTorch version meets requirements (>=2.6.0)
torch_version = tuple(map(int, torch.__version__.split('.')[:2]))
if torch_version < (2, 6):
    print(f"⚠ Warning: PyTorch {torch.__version__} may not meet requirements (>=2.6.0)")
    print("Consider upgrading: !pip install torch>=2.6.0 --upgrade")
else:
    print("✓ PyTorch version meets requirements")


In [ ]:
# Install required packages
# Core ML libraries
%pip install transformers>=4.35.0,<5.0.0 --quiet
%pip install safetensors>=0.4.0 --quiet
%pip install datasets>=2.12.0 --quiet

# ML utilities
%pip install numpy>=1.24.0,<2.0.0 --quiet
%pip install pandas>=2.0.0 --quiet
%pip install scikit-learn>=1.3.0 --quiet

# Utilities
%pip install pyyaml>=6.0 --quiet
%pip install tqdm>=4.65.0 --quiet
%pip install seqeval>=1.2.2 --quiet
%pip install sentencepiece>=0.1.99 --quiet

# Experiment tracking
%pip install mlflow --quiet
%pip install optuna --quiet

# ONNX support
%pip install onnxruntime --quiet
%pip install onnx>=1.16.0 --quiet
%pip install onnxscript>=0.1.0 --quiet

print("✓ All dependencies installed")


## Step 3: Setup Paths and Import Paths

Configure Python paths and verify Colab environment detection.


In [ ]:
# Environment detection and platform configuration
import os
from pathlib import Path

# Detect execution environment
IN_COLAB = "COLAB_GPU" in os.environ or "COLAB_TPU" in os.environ
IN_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ

# Set platform-specific constants
if IN_COLAB:
    PLATFORM = "colab"
    BASE_DIR = Path("/content")
    BACKUP_ENABLED = True
    print("✓ Detected: Google Colab environment")
elif IN_KAGGLE:
    PLATFORM = "kaggle"
    BASE_DIR = Path("/kaggle/working")
    BACKUP_ENABLED = False  # Kaggle outputs are automatically persisted
    print("✓ Detected: Kaggle environment")
else:
    raise EnvironmentError(
        "This notebook requires Google Colab or Kaggle environment.\n"
        "Colab: Set runtime type to GPU/TPU\n"
        "Kaggle: Ensure KAGGLE_KERNEL_RUN_TYPE is set"
    )

print(f"Platform: {PLATFORM}")
print(f"Base directory: {BASE_DIR}")
print(f"Backup enabled: {BACKUP_ENABLED}")


In [ ]:
import os
import sys
from pathlib import Path

# Setup paths (ROOT_DIR should be set in Cell 2)
# If not, set it here
if 'ROOT_DIR' not in globals():
    if IN_COLAB:
        ROOT_DIR = Path("/content/resume-ner-azureml")
    elif IN_KAGGLE:
        ROOT_DIR = Path("/kaggle/working/resume-ner-azureml")
    else:
        ROOT_DIR = Path("/content/resume-ner-azureml")  # Default to Colab path

SRC_DIR = ROOT_DIR / "src"
CONFIG_DIR = ROOT_DIR / "config"
NOTEBOOK_DIR = ROOT_DIR / "notebooks"

# Add to Python path
sys.path.insert(0, str(ROOT_DIR))
sys.path.insert(0, str(SRC_DIR))

print("Notebook directory:", NOTEBOOK_DIR)
print("Project root:", ROOT_DIR)
print("Source directory:", SRC_DIR)
print("Config directory:", CONFIG_DIR)
print("Platform:", PLATFORM if 'PLATFORM' in globals() else "unknown")
print("In Colab:", IN_COLAB if 'IN_COLAB' in globals() else False)
print("In Kaggle:", IN_KAGGLE if 'IN_KAGGLE' in globals() else False)


## Step 4: Mount Google Drive

Mount Google Drive to enable checkpoint persistence across Colab sessions. Checkpoints will be automatically saved to Drive after training completes.


In [ ]:
# Helper functions for checkpoint backup/restore (platform-aware)
import shutil
from pathlib import Path
from typing import Optional

def backup_to_drive(source_path: Path, backup_name: str, is_directory: bool = False) -> bool:
    """
    Backup a file or directory to Google Drive if available.
    
    Args:
        source_path: Path to the file or directory to backup
        backup_name: Name for the backup (will be placed in DRIVE_BACKUP_DIR)
        is_directory: True if backing up a directory, False for a file
    
    Returns:
        True if backup was successful, False if backup is not available or failed
    """
    if not BACKUP_ENABLED or not DRIVE_BACKUP_DIR:
        return False
    
    if not source_path.exists():
        print(f"⚠ Warning: Source path does not exist: {source_path}")
        return False
    
    backup_path = DRIVE_BACKUP_DIR / backup_name
    
    try:
        if is_directory:
            # Remove existing backup if it exists
            if backup_path.exists():
                shutil.rmtree(backup_path)
            shutil.copytree(source_path, backup_path)
        else:
            shutil.copy2(source_path, backup_path)
        
        print(f"✓ Backed up to Google Drive: {backup_path}")
        return True
    except Exception as e:
        print(f"⚠ Warning: Backup failed: {e}")
        return False

def restore_from_drive(backup_name: str, target_path: Path, is_directory: bool = False) -> bool:
    """
    Restore a file or directory from Google Drive if available.
    
    Args:
        backup_name: Name of the backup in DRIVE_BACKUP_DIR
        target_path: Path where the restored file/directory should be placed
        is_directory: True if restoring a directory, False for a file
    
    Returns:
        True if restore was successful, False if backup is not available or failed
    """
    if not BACKUP_ENABLED or not DRIVE_BACKUP_DIR:
        return False
    
    backup_path = DRIVE_BACKUP_DIR / backup_name
    
    if not backup_path.exists():
        return False
    
    try:
        if is_directory:
            # Create parent directory if needed
            target_path.parent.mkdir(parents=True, exist_ok=True)
            shutil.copytree(backup_path, target_path)
        else:
            target_path.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(backup_path, target_path)
        
        print(f"✓ Restored from Google Drive: {target_path}")
        return True
    except Exception as e:
        print(f"⚠ Warning: Restore failed: {e}")
        return False

print("✓ Backup/restore helper functions defined")


In [ ]:
from pathlib import Path

# Mount Google Drive (Colab only - Kaggle doesn't need this)
DRIVE_BACKUP_DIR = None

if IN_COLAB:
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        DRIVE_BACKUP_DIR = Path("/content/drive/MyDrive/resume-ner-checkpoints")
        DRIVE_BACKUP_DIR.mkdir(parents=True, exist_ok=True)
        print(f"✓ Google Drive mounted")
        print(f"✓ Checkpoint backup directory: {DRIVE_BACKUP_DIR}")
        print(f"\nNote: Checkpoints will be automatically saved to this directory after training completes.")
    except ImportError:
        print("⚠ Warning: google.colab.drive not available. Backup to Google Drive will be disabled.")
        BACKUP_ENABLED = False
elif IN_KAGGLE:
    print("✓ Kaggle environment detected - outputs are automatically persisted (no Drive mount needed)")
else:
    print("⚠ Warning: Unknown environment. Backup to Google Drive will be disabled.")


## Step P1-3.1: Load Centralized Configs

Load and validate all configuration files. Configs are immutable and will be logged with each job for reproducibility.


### Step P1-3.1.1: Define Constants

Define constants for file and directory names used throughout the notebook. Benchmark settings come from centralized config, not hard-coded here.


In [ ]:
# Import constants from centralized module
from orchestration import (
    METRICS_FILENAME,
    BENCHMARK_FILENAME,
    CHECKPOINT_DIRNAME,
    OUTPUTS_DIRNAME,
    MLRUNS_DIRNAME,
    DEFAULT_RANDOM_SEED,
    DEFAULT_K_FOLDS,
)

# Note: Benchmark settings (batch_sizes, iterations, etc.) come from configs["benchmark"]


### Step P1-3.1.2: Define Helper Functions

Reusable helper functions following DRY principle for common operations.


In [ ]:
# Import helper functions from consolidated modules (DRY principle)
from typing import List, Optional
from orchestration import (
    build_mlflow_experiment_name,
    setup_mlflow_for_stage,
    run_benchmarking,
)
from shared import verify_output_file

# Wrapper function for run_benchmarking that uses notebook-specific paths
def run_benchmarking_local(
    checkpoint_dir: Path,
    test_data_path: Path,
    output_path: Path,
    batch_sizes: List[int],
    iterations: int,
    warmup_iterations: int,
    max_length: int = 512,
    device: Optional[str] = None,
) -> bool:
    """
    Run benchmarking on a model checkpoint (local notebook wrapper).
    
    This is a thin wrapper around orchestration.benchmark_utils.run_benchmarking
    that automatically uses the notebook's SRC_DIR and ROOT_DIR.
    
    Args:
        checkpoint_dir: Path to checkpoint directory.
        test_data_path: Path to test data JSON file.
        output_path: Path to output benchmark.json file.
        batch_sizes: List of batch sizes to test.
        iterations: Number of iterations per batch size.
        warmup_iterations: Number of warmup iterations.
        max_length: Maximum sequence length.
        device: Device to use (None = auto-detect).
    
    Returns:
        True if successful, False otherwise.
    """
    return run_benchmarking(
        checkpoint_dir=checkpoint_dir,
        test_data_path=test_data_path,
        output_path=output_path,
        batch_sizes=batch_sizes,
        iterations=iterations,
        warmup_iterations=warmup_iterations,
        max_length=max_length,
        device=device,
        project_root=ROOT_DIR,
    )


In [ ]:
from pathlib import Path
from typing import Any, Dict

from orchestration import EXPERIMENT_NAME
from orchestration.config_loader import (
    ExperimentConfig,
    compute_config_hashes,
    create_config_metadata,
    load_all_configs,
    load_experiment_config,
    snapshot_configs,
    validate_config_immutability,
)

# P1-3.1: Load Centralized Configs (local-only)
# Mirrors the Azure orchestration notebook, but does not create an Azure ML client.

if not CONFIG_DIR.exists():
    raise FileNotFoundError(f"Config directory not found: {CONFIG_DIR}")

experiment_config: ExperimentConfig = load_experiment_config(CONFIG_DIR, EXPERIMENT_NAME)
configs: Dict[str, Any] = load_all_configs(experiment_config)
config_hashes = compute_config_hashes(configs)
config_metadata = create_config_metadata(configs, config_hashes)

# Immutable snapshots for runtime mutation checks
original_configs = snapshot_configs(configs)
validate_config_immutability(configs, original_configs)

print(f"Loaded experiment: {experiment_config.name}")
print("Loaded config domains:", sorted(configs.keys()))
print("Config hashes:", config_hashes)
print("Config metadata:", config_metadata)

# Get dataset path from data config (centralized configuration)
# The local_path in the data config is relative to the config directory
data_config = configs["data"]
local_path_str = data_config.get("local_path", "../dataset")
DATASET_LOCAL_PATH = (CONFIG_DIR / local_path_str).resolve()

# Check if seed-based dataset structure (for dataset_tiny with seed subdirectories)
seed = data_config.get("seed")
if seed is not None and "dataset_tiny" in str(DATASET_LOCAL_PATH):
    DATASET_LOCAL_PATH = DATASET_LOCAL_PATH / f"seed{seed}"

print(f"Dataset path (from data config): {DATASET_LOCAL_PATH}")
if seed is not None:
    print(f"Using seed: {seed}")


## Step P1-3.2: Verify Local Dataset

Verify that the dataset directory (specified by `local_path` in the data config) exists and contains the required files. The dataset path is loaded from the centralized data configuration in Step P1-3.1.


In [ ]:
# P1-3.2: Verify Local Dataset
# The dataset path comes from the data config's local_path field (loaded in Step P1-3.1).
# This ensures the dataset location is controlled by centralized configuration.
# Note: train.json is required, but validation.json is optional (matches training script behavior).

REQUIRED_FILE = "train.json"
OPTIONAL_FILE = "validation.json"

if not DATASET_LOCAL_PATH.exists():
    raise FileNotFoundError(
        f"Dataset directory not found: {DATASET_LOCAL_PATH}\n"
        f"This path comes from the data config's 'local_path' field.\n"
        f"If you need to create the dataset, run the notebook: notebooks/00_make_tiny_dataset.ipynb"
    )

# Check required file
train_file = DATASET_LOCAL_PATH / REQUIRED_FILE
if not train_file.exists():
    raise FileNotFoundError(
        f"Required dataset file not found: {train_file}\n"
        f"This path comes from the data config's 'local_path' field.\n"
        f"If you need to create it, run the notebook: notebooks/00_make_tiny_dataset.ipynb"
    )

# Check optional file
val_file = DATASET_LOCAL_PATH / OPTIONAL_FILE
has_validation = val_file.exists()

print(f"✓ Dataset directory found: {DATASET_LOCAL_PATH}")
print(f"  (from data config: {data_config.get('name', 'unknown')} v{data_config.get('version', 'unknown')})")

train_size = train_file.stat().st_size
print(f"  ✓ {REQUIRED_FILE} ({train_size:,} bytes)")

if has_validation:
    val_size = val_file.stat().st_size
    print(f"  ✓ {OPTIONAL_FILE} ({val_size:,} bytes)")
else:
    print(f"  ⚠ {OPTIONAL_FILE} not found (optional - training will proceed without validation set)")


## Step P1-3.2.1: Optional Train/Test Split

**Optional step**: Create a train/test split if `test.json` is missing. This is useful when you only have `train.json` and `validation.json` and want to create a separate test set.

**⚠ WARNING**: This will overwrite `train.json` with the split version. Only enable if you want to create a permanent train/test split.


In [ ]:
# Optional: create train/test split if test.json is missing
# WARNING: This will overwrite train.json with the split version
# Only enable if you want to create a permanent train/test split
import json
from pathlib import Path
from typing import List, Dict, Any, Tuple, Optional

from training.data import split_train_test, save_split_files

CREATE_TEST_SPLIT = False  # Set True to create test.json when absent (WARNING: overwrites train.json)

train_file = DATASET_LOCAL_PATH / "train.json"
val_file = DATASET_LOCAL_PATH / "validation.json"
test_file = DATASET_LOCAL_PATH / "test.json"

if CREATE_TEST_SPLIT and not test_file.exists():
    # Backup original train.json before overwriting
    backup_file = DATASET_LOCAL_PATH / "train.json.backup"
    if train_file.exists() and not backup_file.exists():
        import shutil
        shutil.copy2(train_file, backup_file)
        print(f"⚠ Backed up original train.json to {backup_file}")
    
    full_dataset = []
    # Start with train data; optionally include validation to maximize coverage
    with open(train_file, "r", encoding="utf-8") as f:
        full_dataset.extend(json.load(f))
    if val_file.exists():
        with open(val_file, "r", encoding="utf-8") as f:
            full_dataset.extend(json.load(f))

    split_cfg = configs.get("data", {}).get("splitting", {})
    train_ratio = split_cfg.get("train_test_ratio", 0.8)
    stratified = split_cfg.get("stratified", False)
    random_seed = split_cfg.get("random_seed", 42)
    entity_types = configs.get("data", {}).get("schema", {}).get("entity_types", [])

    print(f"Creating train/test split (train_ratio={train_ratio}, stratified={stratified})...")
    print(f"⚠ WARNING: This will overwrite train.json with {int(len(full_dataset) * train_ratio)} samples")
    
    new_train, new_test = split_train_test(
        dataset=full_dataset,
        train_ratio=train_ratio,
        stratified=stratified,
        random_seed=random_seed,
        entity_types=entity_types,
    )

    save_split_files(DATASET_LOCAL_PATH, new_train, new_test)
    print(f"✓ Wrote train.json ({len(new_train)}) and test.json ({len(new_test)})")
elif test_file.exists():
    print(f"✓ Found existing test.json at {test_file}")
else:
    print("⚠ test.json not found. Set CREATE_TEST_SPLIT=True to generate a split.")


## Step P1-3.3: Setup Local Environment

Verify GPU availability, set up MLflow tracking (local file store), and check that key dependencies are installed. This step ensures the local environment is ready for training.


In [ ]:
import sys
import torch

DEFAULT_DEVICE = "cuda"

env_config = configs["env"]
device_type = env_config.get("compute", {}).get("device", DEFAULT_DEVICE)

if device_type == "cuda" and not torch.cuda.is_available():
    raise RuntimeError(
        "CUDA device requested but not available. "
        "In Colab, ensure you've selected a GPU runtime: Runtime > Change runtime type > GPU"
    )


In [ ]:
import mlflow

MLFLOW_DIR = "mlruns"
mlflow_tracking_path = ROOT_DIR / MLFLOW_DIR
mlflow_tracking_path.mkdir(exist_ok=True)

# Convert path to file:// URI format for MLflow
mlflow_tracking_uri = mlflow_tracking_path.as_uri()
mlflow.set_tracking_uri(mlflow_tracking_uri)


In [ ]:
try:
    import transformers
    import optuna
except ImportError as e:
    raise ImportError(f"Required package not installed: {e}")

REQUIRED_PACKAGES = {
    "torch": torch,
    "transformers": transformers,
    "mlflow": mlflow,
    "optuna": optuna,
}

for name, module in REQUIRED_PACKAGES.items():
    if not hasattr(module, "__version__"):
        raise ImportError(f"Required package '{name}' is not properly installed")


## Step P1-3.4: The Dry Run

Run a minimal HPO sweep to validate the training pipeline works correctly before launching the full HPO sweep. Uses the smoke HPO configuration with reduced trials.


In [ ]:
from pathlib import Path
import importlib.util
from orchestration import STAGE_SMOKE

# Import local_sweeps directly to avoid triggering Azure ML imports in __init__.py
local_sweeps_spec = importlib.util.spec_from_file_location(
    "local_sweeps", SRC_DIR / "orchestration" / "jobs" / "local_sweeps.py"
)
local_sweeps = importlib.util.module_from_spec(local_sweeps_spec)
local_sweeps_spec.loader.exec_module(local_sweeps)
run_local_hpo_sweep = local_sweeps.run_local_hpo_sweep

TRAINING_SCRIPT_PATH = SRC_DIR / "train.py"
DRY_RUN_OUTPUT_DIR = ROOT_DIR / "outputs" / "dry_run"

if not TRAINING_SCRIPT_PATH.exists():
    raise FileNotFoundError(f"Training script not found: {TRAINING_SCRIPT_PATH}")


In [ ]:
hpo_config = configs["hpo"]
train_config = configs["train"]
backbone_values = hpo_config["search_space"]["backbone"]["values"]

dry_run_studies = {}

for backbone in backbone_values:
    mlflow_experiment_name = f"{experiment_config.name}-{STAGE_SMOKE}-{backbone}"
    backbone_output_dir = DRY_RUN_OUTPUT_DIR / backbone
    
    study = run_local_hpo_sweep(
        dataset_path=str(DATASET_LOCAL_PATH),
        config_dir=CONFIG_DIR,
        backbone=backbone,
        hpo_config=hpo_config,
        train_config=train_config,
        output_dir=backbone_output_dir,
        mlflow_experiment_name=mlflow_experiment_name,
    )
    
    dry_run_studies[backbone] = study


In [ ]:
# Use hpo_config from configs (loaded in cell 18 for dry run)
dry_run_hpo_config = configs["hpo"]
objective_metric = dry_run_hpo_config['objective']['metric']

for backbone, study in dry_run_studies.items():
    if study.trials:
        best_trial = study.best_trial
        print(f"{backbone}: {len(study.trials)} trials completed")
        print(f"  Best {objective_metric}: {best_trial.value:.4f}")
        print(f"  Best params: {best_trial.params}")
    else:
        print(f"{backbone}: No trials completed")


## Step P1-3.5.1: The Sweep (HPO) - Local with Optuna

Run the full hyperparameter optimization sweep using Optuna to systematically search for the best model configuration. Uses the production HPO configuration with more trials than the dry run.

**Note on K-Fold Cross-Validation:**
- When k-fold CV is enabled (`k_fold.enabled: true`), each trial trains **k models** (one per fold) and returns the **average metric** across folds
- The number of **trials** is controlled by `sampling.max_trials` (e.g., 2 trials in smoke.yaml)
- With k=5 folds and 2 trials: **2 trials × 5 folds = 10 model trainings total**
- K-fold CV provides more robust hyperparameter evaluation but increases compute time (k× per trial)


In [ ]:
from pathlib import Path
import importlib.util
from orchestration import STAGE_HPO

# Import local_sweeps directly to avoid triggering Azure ML imports in __init__.py
local_sweeps_spec = importlib.util.spec_from_file_location(
    "local_sweeps", SRC_DIR / "orchestration" / "jobs" / "local_sweeps.py"
)
local_sweeps = importlib.util.module_from_spec(local_sweeps_spec)
local_sweeps_spec.loader.exec_module(local_sweeps)
run_local_hpo_sweep = local_sweeps.run_local_hpo_sweep

# Constants are imported from orchestration module (see Step P1-3.1.1)
HPO_OUTPUT_DIR = ROOT_DIR / "outputs" / "hpo"
HPO_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
# Use HPO config already loaded in configs (from Step P1-3.1)
# Following DRY principle - don't reload configs that are already available
hpo_config = configs["hpo"]
train_config = configs["train"]
backbone_values = hpo_config["search_space"]["backbone"]["values"]


## Step P1-3.5.0: Setup K-Fold Splits and Google Drive Backup for HPO Trials

**K-Fold Cross-Validation Setup**: If k-fold CV is enabled in the HPO config, create and save fold splits before starting the sweep.

**Colab-specific feature**: Configure automatic backup of each HPO trial to Google Drive immediately after completion. This prevents data loss if the Colab session disconnects during long-running hyperparameter optimization sweeps.

Each trial's results (including `metrics.json` and checkpoint) are automatically backed up to Google Drive as soon as the trial completes, ensuring no progress is lost even if the session times out.


In [ ]:
from training.cv_utils import create_kfold_splits, save_fold_splits, validate_splits
from training.data import load_dataset

# Setup k-fold splits if enabled
k_fold_config = hpo_config.get("k_fold", {})
k_folds_enabled = k_fold_config.get("enabled", False)
fold_splits_file = None

if k_folds_enabled:
    n_splits = k_fold_config.get("n_splits", DEFAULT_K_FOLDS)
    random_seed = k_fold_config.get("random_seed", DEFAULT_RANDOM_SEED)
    shuffle = k_fold_config.get("shuffle", True)
    stratified = k_fold_config.get("stratified", False)
    entity_types = configs.get("data", {}).get("schema", {}).get("entity_types", [])
    
    print(f"Setting up {n_splits}-fold cross-validation splits...")
    full_dataset = load_dataset(str(DATASET_LOCAL_PATH))
    train_data = full_dataset.get("train", [])
    
    fold_splits = create_kfold_splits(
        dataset=train_data,
        k=n_splits,
        random_seed=random_seed,
        shuffle=shuffle,
        stratified=stratified,
        entity_types=entity_types,
    )
    
    # Optional validation to ensure rare entities appear across folds
    validate_splits(train_data, fold_splits, entity_types=entity_types)
    
    HPO_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    fold_splits_file = HPO_OUTPUT_DIR / "fold_splits.json"
    save_fold_splits(
        fold_splits,
        fold_splits_file,
        metadata={
            "k": n_splits,
            "random_seed": random_seed,
            "shuffle": shuffle,
            "stratified": stratified,
            "dataset_path": str(DATASET_LOCAL_PATH),
        }
    )
    print(f"✓ K-fold splits saved to: {fold_splits_file}")
else:
    print("K-fold CV disabled - using single train/validation split")


In [ ]:
# Colab-specific: Google Drive backup callback for HPO trials
# This ensures trial results are persisted immediately after completion to prevent data loss
# if the Colab session disconnects.

from pathlib import Path
from typing import Any, Optional
import optuna
from orchestration import METRICS_FILENAME


class DriveBackupTrialCallback:
    """Optuna callback to backup trial results to Google Drive after each trial completes.
    
    This callback is Colab-specific and automatically backs up each completed trial's
    directory (including metrics.json and checkpoint) to Google Drive to prevent data
    loss if the Colab session disconnects.
    
    Attributes:
        output_base_dir: Base directory where trials are saved (e.g., HPO_OUTPUT_DIR / backbone)
        backbone: Model backbone name for organizing backups in Google Drive
    """
    
    def __init__(self, output_base_dir: Path, backbone: str) -> None:
        """
        Initialize the backup callback.
        
        Args:
            output_base_dir: Base directory where trials are saved.
            backbone: Model backbone name for organizing backups.
        """
        self.output_base_dir = output_base_dir
        self.backbone = backbone
    
    def __call__(self, study: optuna.Study, trial: optuna.trial.FrozenTrial) -> None:
        """Execute backup after trial completes.
        
        Args:
            study: Optuna study object (unused but required by callback interface).
            trial: Completed trial object containing trial number and state.
        """
        if trial.state != optuna.trial.TrialState.COMPLETE:
            return
        
        if not BACKUP_ENABLED or not DRIVE_BACKUP_DIR:
            return
        
        trial_output_dir = self._find_trial_directory(trial.number)
        if not trial_output_dir or not self._trial_completed_successfully(trial_output_dir):
            return
        
        backup_name = f"hpo_{self.backbone}_trial_{trial.number}"
        success = backup_to_drive(
            source_path=trial_output_dir,
            backup_name=backup_name,
            is_directory=True
        )
        
        if success:
            print(f"  ✓ Trial {trial.number} backed up to Google Drive")
        else:
            print(f"  ⚠ Warning: Failed to backup trial {trial.number}")
    
    def _find_trial_directory(self, trial_number: int) -> Optional[Path]:
        """Find the trial output directory, handling both regular and CV trials.
        
        Args:
            trial_number: Trial number to find directory for.
            
        Returns:
            Path to trial directory, or None if not found.
        """
        trial_dir_name = f"trial_{trial_number}"
        trial_output_dir = self.output_base_dir / trial_dir_name
        
        if trial_output_dir.exists():
            return trial_output_dir
        
        fold_dirs = list(self.output_base_dir.glob(f"trial_{trial_number}_fold*"))
        if fold_dirs:
            fold_dirs.sort(key=lambda x: int(x.name.split('fold')[-1]) if 'fold' in x.name else 0)
            return fold_dirs[-1]
        
        print(f"⚠ Warning: Trial {trial_number} output directory not found")
        return None
    
    def _trial_completed_successfully(self, trial_output_dir: Path) -> bool:
        """Check if trial completed successfully by verifying metrics.json exists.
        
        Args:
            trial_output_dir: Path to trial output directory.
            
        Returns:
            True if metrics.json exists, False otherwise.
        """
        metrics_file = trial_output_dir / METRICS_FILENAME
        if not metrics_file.exists():
            print(f"⚠ Warning: Trial {trial_output_dir.name} metrics.json not found, skipping backup")
            return False
        return True



In [ ]:
# Colab-specific wrapper function that adds Google Drive backup to HPO sweep
# This recreates the study setup from run_local_hpo_sweep but adds our backup callback

from typing import Any, Optional


def run_local_hpo_sweep_with_backup(
    dataset_path: str,
    config_dir: Path,
    backbone: str,
    hpo_config: dict,
    train_config: dict,
    output_dir: Path,
    mlflow_experiment_name: str,
    k_folds: Optional[int] = None,
    fold_splits_file: Optional[Path] = None,
) -> Any:
    """Run HPO sweep with automatic Google Drive backup after each trial.
    
    This is a Colab-specific wrapper around run_local_hpo_sweep that adds automatic
    backup of each trial to Google Drive immediately after completion. This prevents
    data loss if the Colab session disconnects during long-running HPO sweeps.
    
    Args:
        dataset_path: Path to dataset directory.
        config_dir: Path to configuration directory.
        backbone: Model backbone name.
        hpo_config: HPO configuration dictionary.
        train_config: Training configuration dictionary.
        output_dir: Base output directory for all trials.
        mlflow_experiment_name: MLflow experiment name.
        k_folds: Number of folds for k-fold CV (None = no CV).
        fold_splits_file: Path to fold splits file (for k-fold CV).
    
    Returns:
        Optuna study object with completed trials.
    """
    optuna_module, _, RandomSampler, _ = local_sweeps._import_optuna()
    
    objective_metric = hpo_config["objective"]["metric"]
    goal = hpo_config["objective"]["goal"]
    direction = "maximize" if goal == "maximize" else "minimize"
    
    pruner = local_sweeps.create_optuna_pruner(hpo_config)
    algorithm = hpo_config["sampling"]["algorithm"].lower()
    sampler = RandomSampler() if algorithm == "random" else RandomSampler()
    
    print(f"\n[HPO] Starting optimization for {backbone} (with Google Drive backup)...")
    study = optuna_module.create_study(
        direction=direction,
        sampler=sampler,
        pruner=pruner,
        study_name=f"hpo_{backbone}",
    )
    
    objective = local_sweeps.create_local_hpo_objective(
        dataset_path=dataset_path,
        config_dir=config_dir,
        backbone=backbone,
        hpo_config=hpo_config,
        train_config=train_config,
        output_base_dir=output_dir,
        mlflow_experiment_name=mlflow_experiment_name,
        objective_metric=objective_metric,
        k_folds=k_folds,
        fold_splits_file=fold_splits_file,
    )
    
    backup_callback = DriveBackupTrialCallback(
        output_base_dir=output_dir,
        backbone=backbone,
    )
    
    def display_metrics_callback(study: Any, trial: Any) -> None:
        """Display additional metrics after trial completes."""
        optuna_module, _, _, _ = local_sweeps._import_optuna()
        if trial.state == optuna_module.trial.TrialState.COMPLETE:
            attrs = trial.user_attrs
            extra_info = []
            
            if "macro_f1_span" in attrs:
                extra_info.append(f"macro-f1-span={attrs['macro_f1_span']:.6f}")
            if "loss" in attrs:
                extra_info.append(f"loss={attrs['loss']:.6f}")
            if "avg_entity_f1" in attrs:
                entity_count = attrs.get("entity_count", "?")
                extra_info.append(f"avg_entity_f1={attrs['avg_entity_f1']:.6f} ({entity_count} entities)")
            
            if extra_info:
                print(f"  Additional metrics: {' | '.join(extra_info)}")
    
    def combined_callback(study: Any, trial: Any) -> None:
        """Combined callback that displays metrics and backs up to Google Drive."""
        display_metrics_callback(study, trial)
        backup_callback(study, trial)
    
    max_trials = hpo_config["sampling"]["max_trials"]
    timeout_minutes = hpo_config["sampling"]["timeout_minutes"]
    timeout_seconds = timeout_minutes * 60
    
    study.optimize(
        objective,
        n_trials=max_trials,
        timeout=timeout_seconds,
        show_progress_bar=True,
        callbacks=[combined_callback],
    )
    
    return study



In [ ]:
hpo_studies = {}
k_folds_param = k_fold_config.get("n_splits", DEFAULT_K_FOLDS) if k_folds_enabled else None

for backbone in backbone_values:
    mlflow_experiment_name = build_mlflow_experiment_name(
        experiment_config.name, STAGE_HPO, backbone
    )
    backbone_output_dir = HPO_OUTPUT_DIR / backbone
    
    # Use Colab-specific wrapper with automatic Google Drive backup
    study = run_local_hpo_sweep_with_backup(
        dataset_path=str(DATASET_LOCAL_PATH),
        config_dir=CONFIG_DIR,
        backbone=backbone,
        hpo_config=hpo_config,
        train_config=train_config,
        output_dir=backbone_output_dir,
        mlflow_experiment_name=mlflow_experiment_name,
        k_folds=k_folds_param,
        fold_splits_file=fold_splits_file,
    )
    
    hpo_studies[backbone] = study


In [ ]:
def extract_cv_statistics(best_trial):
    if not hasattr(best_trial, "user_attrs"):
        return None
    cv_mean = best_trial.user_attrs.get("cv_mean")
    cv_std = best_trial.user_attrs.get("cv_std")
    return (cv_mean, cv_std) if cv_mean is not None else None

objective_metric = hpo_config['objective']['metric']

for backbone, study in hpo_studies.items():
    if not study.trials:
        continue
    
    best_trial = study.best_trial
    cv_stats = extract_cv_statistics(best_trial)
    
    print(f"{backbone}: {len(study.trials)} trials completed")
    print(f"  Best {objective_metric}: {best_trial.value:.4f}")
    print(f"  Best params: {best_trial.params}")
    
    if cv_stats:
        cv_mean, cv_std = cv_stats
        print(f"  CV Statistics: Mean: {cv_mean:.4f} ± {cv_std:.4f}")


## Step P1-3.5.2: Benchmarking Best Trials

Benchmark the best trial from each backbone to measure actual inference performance. This provides real latency data that replaces parameter-count proxies in model selection, enabling more accurate speed comparisons.

**Workflow:**
1. Identify best trial per backbone (from HPO results)
2. Run benchmarking on each best trial checkpoint
3. Save benchmark results as `benchmark.json` in trial directories
4. Model selection will automatically use this data when available


In [ ]:
from orchestration.jobs.local_selection import load_best_trial_from_disk
import json

# Load benchmark config (if available)
benchmark_config = configs.get("benchmark", {})
benchmark_settings = benchmark_config.get("benchmarking", {})

# Get benchmark parameters from config or use defaults
benchmark_batch_sizes = benchmark_settings.get("batch_sizes", [1, 8, 16])
benchmark_iterations = benchmark_settings.get("iterations", 100)
benchmark_warmup = benchmark_settings.get("warmup_iterations", 10)
benchmark_max_length = benchmark_settings.get("max_length", 512)
benchmark_device = benchmark_settings.get("device")

# Get test data path from benchmark config or data config
test_data_path_str = benchmark_settings.get("test_data")
if test_data_path_str:
    test_data_path = (CONFIG_DIR / test_data_path_str).resolve()
else:
    # Fallback to dataset directory
    test_data_path = DATASET_LOCAL_PATH / "test.json"

if not test_data_path.exists():
    print(f"Warning: Test data not found at {test_data_path}")
    print("Benchmarking will be skipped. Model selection will use parameter proxy.")
    test_data_path = None

# Identify best trials per backbone
objective_metric = hpo_config["objective"]["metric"]
best_trials = {}

for backbone in backbone_values:
    best_trial_info = load_best_trial_from_disk(
        HPO_OUTPUT_DIR,
        backbone,
        objective_metric
    )
    if best_trial_info:
        best_trials[backbone] = best_trial_info
        print(f"{backbone}: Best trial is {best_trial_info['trial_name']} "
              f"({objective_metric}={best_trial_info['accuracy']:.4f})")


In [ ]:
# Run benchmarking on best trials
if test_data_path and test_data_path.exists():
    benchmark_results = {}
    
    for backbone, trial_info in best_trials.items():
        trial_dir = Path(trial_info["trial_dir"])
        checkpoint_dir = trial_dir / CHECKPOINT_DIRNAME
        benchmark_output = trial_dir / BENCHMARK_FILENAME
        
        if not checkpoint_dir.exists():
            print(f"Warning: Checkpoint not found for {backbone} {trial_info['trial_name']}")
            continue
        
        print(f"\nBenchmarking {backbone} ({trial_info['trial_name']})...")
        
        success = run_benchmarking_local(
            checkpoint_dir=checkpoint_dir,
            test_data_path=test_data_path,
            output_path=benchmark_output,
            batch_sizes=benchmark_batch_sizes,
            iterations=benchmark_iterations,
            warmup_iterations=benchmark_warmup,
            max_length=benchmark_max_length,
            device=benchmark_device,
        )
        
        if success:
            benchmark_results[backbone] = benchmark_output
            print(f"✓ Benchmark completed: {benchmark_output}")
        else:
            print(f"✗ Benchmark failed for {backbone}")
    
    print(f"\n✓ Benchmarking complete. {len(benchmark_results)}/{len(best_trials)} trials benchmarked.")
else:
    print("Skipping benchmarking (test data not available)")


In [ ]:
# Verify benchmark files were created
if test_data_path and test_data_path.exists():
    for backbone, trial_info in best_trials.items():
        trial_dir = Path(trial_info["trial_dir"])
        benchmark_file = trial_dir / BENCHMARK_FILENAME
        
        if benchmark_file.exists():
            with open(benchmark_file, "r") as f:
                benchmark_data = json.load(f)
            batch_1_latency = benchmark_data.get("batch_1", {}).get("mean_ms", "N/A")
            print(f"{backbone}: benchmark.json exists (batch_1 latency: {batch_1_latency} ms)")
        else:
            print(f"{backbone}: benchmark.json not found (will use parameter proxy)")


## Step P1-3.6: Best Configuration Selection (Automated)

Programmatically select the best configuration from all HPO sweep runs across all backbone models. The best configuration is determined by the objective metric specified in the HPO config.


In [ ]:
from pathlib import Path
import importlib.util
from shared.json_cache import save_json

# Import local_selection directly to avoid triggering Azure ML imports in __init__.py
local_selection_spec = importlib.util.spec_from_file_location(
    "local_selection", SRC_DIR / "orchestration" / "jobs" / "local_selection.py"
)
local_selection = importlib.util.module_from_spec(local_selection_spec)
local_selection_spec.loader.exec_module(local_selection)
select_best_configuration_across_studies = local_selection.select_best_configuration_across_studies

BEST_CONFIG_CACHE_FILE = ROOT_DIR / "notebooks" / "best_configuration_cache.json"


In [ ]:
dataset_version = data_config.get("version", "unknown")

# Select best configuration with accuracy-speed tradeoff
# Supports both in-memory studies and disk-based selection
# Uses threshold from hpo_config["selection"] if configured

# Option 1: Use in-memory studies (if notebook still running)
if 'hpo_studies' in locals() and hpo_studies:
    best_configuration = select_best_configuration_across_studies(
        studies=hpo_studies,
        hpo_config=hpo_config,
        dataset_version=dataset_version,
        # Uses accuracy_threshold from hpo_config["selection"] if set
    )
else:
    # Option 2: Load from disk (works after notebook restart)
    HPO_OUTPUT_DIR = ROOT_DIR / "outputs" / "hpo"
    best_configuration = select_best_configuration_across_studies(
        studies=None,  # No in-memory studies
        hpo_config=hpo_config,
        dataset_version=dataset_version,
        hpo_output_dir=HPO_OUTPUT_DIR,  # Load from saved metrics.json files
        # Uses accuracy_threshold from hpo_config["selection"] if set
    )


In [ ]:
save_json(BEST_CONFIG_CACHE_FILE, best_configuration)

print(f"Best configuration selected:")
print(f"  Backbone: {best_configuration.get('backbone')}")
print(f"  Trial: {best_configuration.get('trial_name')}")
print(f"  Best {hpo_config['objective']['metric']}: {best_configuration.get('selection_criteria', {}).get('best_value'):.4f}")

# Show selection reasoning (if available)
selection_criteria = best_configuration.get('selection_criteria', {})
if 'reason' in selection_criteria:
    print(f"  Selection reason: {selection_criteria['reason']}")
if 'accuracy_diff_from_best' in selection_criteria:
    print(f"  Accuracy difference from best: {selection_criteria['accuracy_diff_from_best']:.4f}")

# Show all candidates (if available)
if 'all_candidates' in selection_criteria:
    print(f"\nAll candidates considered:")
    for c in selection_criteria['all_candidates']:
        marker = "✓" if c['backbone'] == best_configuration.get('backbone') else " "
        print(f"  {marker} {c['backbone']}: acc={c['accuracy']:.4f}, speed={c['speed_score']:.2f}x")

print(f"\nSaved to: {BEST_CONFIG_CACHE_FILE}")


## Step P1-3.7: Final Training (Post-HPO, Single Run)

Train the final production model using the best configuration from HPO with stable, controlled conditions. This uses the full training epochs (no early stopping) and the best hyperparameters found during HPO.

**Note**: After training completes, the checkpoint will be automatically backed up to Google Drive for persistence.


In [ ]:
from pathlib import Path
import os
import sys
import subprocess
import mlflow
from shared.json_cache import load_json, save_json
from orchestration import STAGE_TRAINING

# Define build_final_training_config locally to avoid importing Azure ML dependencies
# This function doesn't use Azure ML, so we can define it here
def build_final_training_config(
    best_config: dict,
    train_config: dict,
    random_seed: int = 42,
) -> dict:
    """
    Build final training configuration by merging best HPO config with train.yaml defaults.
    """
    hyperparameters = best_config.get("hyperparameters", {})
    training_defaults = train_config.get("training", {})
    
    return {
        "backbone": best_config["backbone"],
        "learning_rate": hyperparameters.get("learning_rate", training_defaults.get("learning_rate", 2e-5)),
        "dropout": hyperparameters.get("dropout", training_defaults.get("dropout", 0.1)),
        "weight_decay": hyperparameters.get("weight_decay", training_defaults.get("weight_decay", 0.01)),
        "batch_size": training_defaults.get("batch_size", 16),
        "epochs": training_defaults.get("epochs", 5),
        "random_seed": random_seed,
        "early_stopping_enabled": False,
        "use_combined_data": True,
        "use_all_data": True,
    }

DEFAULT_RANDOM_SEED = 42
BEST_CONFIG_CACHE_FILE = ROOT_DIR / "notebooks" / "best_configuration_cache.json"
FINAL_TRAINING_OUTPUT_DIR = ROOT_DIR / "outputs" / "final_training"


In [ ]:
best_configuration = load_json(BEST_CONFIG_CACHE_FILE, default=None)

if best_configuration is None:
    raise FileNotFoundError(
        f"Best configuration cache not found: {BEST_CONFIG_CACHE_FILE}\n"
        f"Please run Step P1-3.6: Best Configuration Selection first."
    )

final_training_config = build_final_training_config(
    best_config=best_configuration,
    train_config=configs["train"],
    random_seed=DEFAULT_RANDOM_SEED,
)


In [ ]:
mlflow_experiment_name = f"{experiment_config.name}-{STAGE_TRAINING}-{final_training_config['backbone']}"
final_output_dir = FINAL_TRAINING_OUTPUT_DIR / final_training_config['backbone']
final_output_dir.mkdir(parents=True, exist_ok=True)

mlflow.set_experiment(mlflow_experiment_name)


In [ ]:
training_script_path = SRC_DIR / "train.py"
training_args = [
    sys.executable,
    str(training_script_path),
    "--data-asset",
    str(DATASET_LOCAL_PATH),
    "--config-dir",
    str(CONFIG_DIR),
    "--backbone",
    final_training_config["backbone"],
    "--learning-rate",
    str(final_training_config["learning_rate"]),
    "--batch-size",
    str(final_training_config["batch_size"]),
    "--dropout",
    str(final_training_config["dropout"]),
    "--weight-decay",
    str(final_training_config["weight_decay"]),
    "--epochs",
    str(final_training_config["epochs"]),
    "--random-seed",
    str(final_training_config["random_seed"]),
    "--early-stopping-enabled",
    str(final_training_config["early_stopping_enabled"]).lower(),
    "--use-combined-data",
    str(final_training_config["use_combined_data"]).lower(),
]


In [ ]:
training_env = os.environ.copy()
training_env["AZURE_ML_OUTPUT_checkpoint"] = str(final_output_dir)

mlflow_tracking_uri = mlflow.get_tracking_uri()
if mlflow_tracking_uri:
    training_env["MLFLOW_TRACKING_URI"] = mlflow_tracking_uri
training_env["MLFLOW_EXPERIMENT_NAME"] = mlflow_experiment_name


In [ ]:
result = subprocess.run(
    training_args,
    cwd=ROOT_DIR,
    env=training_env,
    capture_output=False,
    text=True,
)

if result.returncode != 0:
    raise RuntimeError(f"Final training failed with return code {result.returncode}")


In [ ]:
import json
import shutil
from pathlib import Path
import os

# Constants are imported from orchestration module (see Step P1-3.1.1)
FINAL_TRAINING_CACHE_FILE = ROOT_DIR / "notebooks" / "final_training_cache.json"

# Check actual checkpoint location
# The training script may save to outputs/checkpoint instead of final_output_dir/checkpoint
actual_checkpoint = ROOT_DIR / "outputs" / "checkpoint"
actual_metrics = ROOT_DIR / "outputs" / METRICS_FILENAME
expected_checkpoint = final_output_dir / "checkpoint"
expected_metrics = final_output_dir / METRICS_FILENAME

print("Checking training completion...")
print(f"  Expected checkpoint: {expected_checkpoint} (exists: {expected_checkpoint.exists()})")
print(f"  Actual checkpoint: {actual_checkpoint} (exists: {actual_checkpoint.exists()})")
print(f"  Expected metrics: {expected_metrics} (exists: {expected_metrics.exists()})")
print(f"  Actual metrics: {actual_metrics} (exists: {actual_metrics.exists()})")

# Determine which checkpoint and metrics to use
checkpoint_source = None
metrics_file = None

if expected_checkpoint.exists() and any(expected_checkpoint.iterdir()):
    checkpoint_source = expected_checkpoint
    print(f"✓ Using expected checkpoint location: {checkpoint_source}")
elif actual_checkpoint.exists() and any(actual_checkpoint.iterdir()):
    checkpoint_source = actual_checkpoint
    print(f"✓ Using actual checkpoint location: {checkpoint_source}")
    # Update final_output_dir to match actual location
    final_output_dir = actual_checkpoint.parent

if expected_metrics.exists():
    metrics_file = expected_metrics
elif actual_metrics.exists():
    metrics_file = actual_metrics

# Load metrics if available
metrics = None
if metrics_file and metrics_file.exists():
    with open(metrics_file, "r") as f:
        metrics = json.load(f)
    print(f"✓ Metrics loaded from: {metrics_file}")
    print(f"  Metrics: {metrics}")
elif checkpoint_source:
    print(f"⚠ Warning: Metrics file not found, but checkpoint exists.")
    metrics = {"status": "completed", "checkpoint_found": True}
else:
    raise FileNotFoundError(
        f"Training completed but no checkpoint found.\n"
        f"  Expected: {expected_checkpoint}\n"
        f"  Actual: {actual_checkpoint}\n"
        f"  Please check training logs for errors."
    )

# Save cache file with actual paths
save_json(FINAL_TRAINING_CACHE_FILE, {
    "output_dir": str(final_output_dir),
    "backbone": final_training_config["backbone"],
    "config": final_training_config,
})

# Backup checkpoint to Google Drive (if available)
if checkpoint_source and checkpoint_source.exists() and any(checkpoint_source.iterdir()):
    backup_to_drive(
        checkpoint_source,
        f"{final_training_config['backbone']}_checkpoint",
        is_directory=True
    )
    
    # Backup cache file to Drive
    backup_to_drive(FINAL_TRAINING_CACHE_FILE, "final_training_cache.json", is_directory=False)
else:
    raise FileNotFoundError(
        f"Checkpoint directory not found or empty.\n"
        f"  Expected: {expected_checkpoint}\n"
        f"  Actual: {actual_checkpoint}\n"
        f"Training may have failed. Please check the training output above."
    )


## Step P1-4: Model Conversion & Optimization

Convert the final training checkpoint to an optimized ONNX model (int8 quantized) for production inference.

**Platform Adapter Note**: The conversion script (`src/convert_to_onnx.py`) uses the platform adapter to automatically handle output paths and logging appropriately for local execution.

**Checkpoint Restoration**: 
- **Google Colab**: If the checkpoint is not found locally (e.g., after a session disconnect), it will be automatically restored from Google Drive.
- **Kaggle**: Checkpoints are automatically persisted in `/kaggle/working/` - no restoration needed.


In [ ]:
from pathlib import Path
import os
import sys
import subprocess
import mlflow
import shutil
from shared.json_cache import load_json

CONVERSION_SCRIPT_PATH = SRC_DIR / "convert_to_onnx.py"
FINAL_TRAINING_CACHE_FILE = ROOT_DIR / "notebooks" / "final_training_cache.json"
CONVERSION_OUTPUT_DIR = ROOT_DIR / "outputs" / "conversion"


In [ ]:
training_cache = load_json(FINAL_TRAINING_CACHE_FILE, default=None)

if training_cache is None:
    # Try to restore from Google Drive (if available)
    if restore_from_drive("final_training_cache.json", FINAL_TRAINING_CACHE_FILE, is_directory=False):
        training_cache = load_json(FINAL_TRAINING_CACHE_FILE, default=None)
    else:
        raise FileNotFoundError(
            f"Final training cache not found locally or in backup.\n"
            f"Please run Step P1-3.7: Final Training first."
        )

# Try to find checkpoint in expected location or actual location
backbone = training_cache["backbone"]
expected_checkpoint_dir = Path(training_cache["output_dir"]) / "checkpoint"
actual_checkpoint_dir = ROOT_DIR / "outputs" / "checkpoint"

print(f"Looking for checkpoint...")
print(f"  Expected: {expected_checkpoint_dir} (exists: {expected_checkpoint_dir.exists()})")
print(f"  Actual: {actual_checkpoint_dir} (exists: {actual_checkpoint_dir.exists()})")

# Determine which checkpoint to use
checkpoint_dir = None
if expected_checkpoint_dir.exists() and any(expected_checkpoint_dir.iterdir()):
    checkpoint_dir = expected_checkpoint_dir
    print(f"✓ Using expected checkpoint location: {checkpoint_dir}")
elif actual_checkpoint_dir.exists() and any(actual_checkpoint_dir.iterdir()):
    checkpoint_dir = actual_checkpoint_dir
    print(f"✓ Using actual checkpoint location: {checkpoint_dir}")
else:
    # Try to restore from Google Drive (if available)
    checkpoint_dir = expected_checkpoint_dir
    if restore_from_drive(f"{backbone}_checkpoint", checkpoint_dir, is_directory=True):
        print(f"✓ Checkpoint restored from backup")
    else:
        raise FileNotFoundError(
            f"Checkpoint directory not found locally or in backup.\n"
            f"  Expected: {expected_checkpoint_dir}\n"
            f"  Actual: {actual_checkpoint_dir}\n"
            f"Please ensure training completed successfully and checkpoint was backed up."
        )

conversion_output_dir = CONVERSION_OUTPUT_DIR / backbone
conversion_output_dir.mkdir(parents=True, exist_ok=True)


In [ ]:
conversion_args = [
    sys.executable,
    str(CONVERSION_SCRIPT_PATH),
    "--checkpoint-path",
    str(checkpoint_dir),
    "--config-dir",
    str(CONFIG_DIR),
    "--backbone",
    backbone,
    "--output-dir",
    str(conversion_output_dir),
    "--quantize-int8",
    "--run-smoke-test",
]


In [ ]:
conversion_env = os.environ.copy()
conversion_env["AZURE_ML_OUTPUT_onnx_model"] = str(conversion_output_dir)

mlflow_tracking_uri = mlflow.get_tracking_uri()
if mlflow_tracking_uri:
    conversion_env["MLFLOW_TRACKING_URI"] = mlflow_tracking_uri


In [ ]:
result = subprocess.run(
    conversion_args,
    cwd=ROOT_DIR,
    env=conversion_env,
    capture_output=False,
    text=True,
)

if result.returncode != 0:
    raise RuntimeError(f"Model conversion failed with return code {result.returncode}")


In [ ]:
from shared.json_cache import save_json
import shutil

ONNX_MODEL_FILENAME = "model_int8.onnx"
FALLBACK_ONNX_MODEL_FILENAME = "model.onnx"
CONVERSION_CACHE_FILE = ROOT_DIR / "notebooks" / "conversion_cache.json"

onnx_model_path = conversion_output_dir / ONNX_MODEL_FILENAME
if not onnx_model_path.exists():
    onnx_model_path = conversion_output_dir / FALLBACK_ONNX_MODEL_FILENAME

if not onnx_model_path.exists():
    raise FileNotFoundError(f"ONNX model not found in {conversion_output_dir}")

print(f"✓ Conversion completed. ONNX model: {onnx_model_path}")

save_json(CONVERSION_CACHE_FILE, {
    "onnx_model_path": str(onnx_model_path),
    "backbone": backbone,
    "checkpoint_dir": str(checkpoint_dir),
})

# Backup ONNX model to Google Drive (if available)
if onnx_model_path.exists():
    backup_to_drive(onnx_model_path, f"{backbone}_model.onnx", is_directory=False)
else:
    print(f"⚠ Warning: ONNX model not found for backup: {onnx_model_path}")

# Backup conversion cache file to Drive
backup_to_drive(CONVERSION_CACHE_FILE, "conversion_cache.json", is_directory=False)
